In [37]:
import os

# List all files in a folder
folder_path = './'
print(os.listdir(folder_path))


['code.ipynb', 'Data Dictionary.xlsx', 'Data_Dictionary.xlsx', 'historical_transactions.csv', 'merchants.csv', 'new_merchant_transactions.csv', 'sample_submission.csv', 'test.csv', 'test_data.csv', 'train.csv', 'train_data.csv']


In [38]:
import numpy as np # linear algebra
import pandas as pd
import numpy as np
import pandas as pd
import os
import time
import warnings
import gc
gc.collect()
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline

from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler

In [39]:
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

# Models

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error

from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from scipy.stats import reciprocal, uniform

from sklearn.model_selection import StratifiedKFold, RepeatedKFold

# Cross-validation
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import cross_validate

# GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#Common data processors
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from scipy import sparse
np.random.seed(123)
gc.collect()

11

In [40]:
file_path = './train.csv'

# 3. Load the CSV into a DataFrame
train = pd.read_csv(file_path)
train.head()







,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [41]:
# Display train set details
print(f'Train Set: {len(train)} rows, {len(train["card_id"].unique())} entries\n')

# Convert specified features to categorical
for feature in ['feature_1', 'feature_2', 'feature_3']:
    train[feature] = train[feature].astype('category')

# Convert target column to float32
train['target'] = train['target'].astype('float32')

# Check for missing values
print(train.isnull().sum())

Train Set: 201917 rows, 201917 entries

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64


In [42]:
file_path = './test.csv'
test = pd.read_csv(file_path)
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [43]:
print(f'Test Set: {len(test)} rows, {len(test["card_id"].unique())} entries\n')

for feature in ['feature_1','feature_2','feature_3']:
    test[feature] = test[feature].astype('category')

print(test.isnull().sum())

Test Set: 123623 rows, 123623 entries

first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64


In [44]:
test['first_active_month'].fillna(test['first_active_month'].mode()[0], inplace=True)
print(test.isnull().sum())



first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64


In [45]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinffo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [46]:

file_path = './historical_transactions.csv'
df_hist = pd.read_csv(file_path)
df_hist.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id']

In [47]:
for feature in df_hist.columns.tolist():
    if feature in ['installments','month_lag','purchase_amount']:
        df_hist[feature] = df_hist[feature].astype('float32')
        df_hist[feature].replace([np.inf, -np.inf], np.nan, inplace=True)
    else:
        df_hist[feature] = df_hist[feature].astype('category')

print(df_hist.isnull().sum())
df_hist.dtypes

authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               178159
merchant_category_id          0
merchant_id              138481
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2652864
state_id                      0
subsector_id                  0
dtype: int64


authorized_flag         category
card_id                 category
city_id                 category
category_1              category
installments             float32
category_3              category
merchant_category_id    category
merchant_id             category
month_lag                float32
purchase_amount          float32
purchase_date           category
category_2              category
state_id                category
subsector_id            category
dtype: object

In [48]:
def fill_na(df):
    for feature in df.columns.tolist():
        if (df[feature].dtype == 'int32') or (df[feature].dtype == 'float32'):
            df[feature].fillna(df[feature].mean(), inplace=True)
        else:
            df[feature].fillna(df[feature].mode()[0], inplace=True)

fill_na(df_hist)
print(df_hist.isnull().sum())

authorized_flag         0
card_id                 0
city_id                 0
category_1              0
installments            0
category_3              0
merchant_category_id    0
merchant_id             0
month_lag               0
purchase_amount         0
purchase_date           0
category_2              0
state_id                0
subsector_id            0
dtype: int64


In [49]:
df_new = pd.read_csv('./new_merchant_transactions.csv')
df_new.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id']

In [50]:
for feature in df_new.columns.tolist():
    if feature in ['installments','month_lag','purchase_amount']:
        df_new[feature] = df_new[feature].astype('float32')
        df_new[feature].replace([np.inf, -np.inf], np.nan, inplace=True)
    else:
        df_new[feature] = df_new[feature].astype('category')

print(df_new.isnull().sum())
df_new.dtypes

authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64


authorized_flag         category
card_id                 category
city_id                 category
category_1              category
installments             float32
category_3              category
merchant_category_id    category
merchant_id             category
month_lag                float32
purchase_amount          float32
purchase_date           category
category_2              category
state_id                category
subsector_id            category
dtype: object

In [51]:
fill_na(df_new)
print(df_new.isnull().sum())

authorized_flag         0
card_id                 0
city_id                 0
category_1              0
installments            0
category_3              0
merchant_category_id    0
merchant_id             0
month_lag               0
purchase_amount         0
purchase_date           0
category_2              0
state_id                0
subsector_id            0
dtype: int64


In [52]:
df_transactions = pd.concat([df_hist, df_new], axis=0)
df_transactions.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id']

In [53]:
df_merchants = pd.read_csv('./merchants.csv')
df_merchants.columns.tolist()

['merchant_id',
 'merchant_group_id',
 'merchant_category_id',
 'subsector_id',
 'numerical_1',
 'numerical_2',
 'category_1',
 'most_recent_sales_range',
 'most_recent_purchases_range',
 'avg_sales_lag3',
 'avg_purchases_lag3',
 'active_months_lag3',
 'avg_sales_lag6',
 'avg_purchases_lag6',
 'active_months_lag6',
 'avg_sales_lag12',
 'avg_purchases_lag12',
 'active_months_lag12',
 'category_4',
 'city_id',
 'state_id',
 'category_2']

In [54]:
for feature in df_merchants.columns.tolist():
    if feature in ['numerical_1','numerical_2',
                   'avg_sales_lag3','avg_purchases_lag3','active_months_lag3',
                   'avg_sales_lag6','avg_purchases_lag6','active_months_lag6',
                   'avg_sales_lag12','avg_purchases_lag12','active_months_lag12',]:
        
        df_merchants[feature] = df_merchants[feature].astype('float32')
        df_merchants[feature].replace([np.inf, -np.inf], np.nan, inplace=True)
    else:
        df_merchants[feature] = df_merchants[feature].astype('category')

print(df_merchants.isnull().sum())
df_merchants.dtypes

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 3
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 3
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                3
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64


merchant_id                    category
merchant_group_id              category
merchant_category_id           category
subsector_id                   category
numerical_1                     float32
numerical_2                     float32
category_1                     category
most_recent_sales_range        category
most_recent_purchases_range    category
avg_sales_lag3                  float32
avg_purchases_lag3              float32
active_months_lag3              float32
avg_sales_lag6                  float32
avg_purchases_lag6              float32
active_months_lag6              float32
avg_sales_lag12                 float32
avg_purchases_lag12             float32
active_months_lag12             float32
category_4                     category
city_id                        category
state_id                       category
category_2                     category
dtype: object

In [55]:
fill_na(df_merchants)
print(df_merchants.isnull().sum())

merchant_id                    0
merchant_group_id              0
merchant_category_id           0
subsector_id                   0
numerical_1                    0
numerical_2                    0
category_1                     0
most_recent_sales_range        0
most_recent_purchases_range    0
avg_sales_lag3                 0
avg_purchases_lag3             0
active_months_lag3             0
avg_sales_lag6                 0
avg_purchases_lag6             0
active_months_lag6             0
avg_sales_lag12                0
avg_purchases_lag12            0
active_months_lag12            0
category_4                     0
city_id                        0
state_id                       0
category_2                     0
dtype: int64


In [56]:
df_trans_merch = pd.merge(df_transactions, df_merchants, on=[col for col in df_transactions.columns.tolist() if col in df_merchants.columns.tolist()], how='inner')
df_trans_merch.columns.tolist()

['authorized_flag',
 'card_id',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id',
 'merchant_group_id',
 'numerical_1',
 'numerical_2',
 'most_recent_sales_range',
 'most_recent_purchases_range',
 'avg_sales_lag3',
 'avg_purchases_lag3',
 'active_months_lag3',
 'avg_sales_lag6',
 'avg_purchases_lag6',
 'active_months_lag6',
 'avg_sales_lag12',
 'avg_purchases_lag12',
 'active_months_lag12',
 'category_4']

In [57]:
train_merge = pd.merge(train, df_trans_merch, on='card_id', how='left')
train_merge.columns.tolist()

['first_active_month',
 'card_id',
 'feature_1',
 'feature_2',
 'feature_3',
 'target',
 'authorized_flag',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id',
 'merchant_group_id',
 'numerical_1',
 'numerical_2',
 'most_recent_sales_range',
 'most_recent_purchases_range',
 'avg_sales_lag3',
 'avg_purchases_lag3',
 'active_months_lag3',
 'avg_sales_lag6',
 'avg_purchases_lag6',
 'active_months_lag6',
 'avg_sales_lag12',
 'avg_purchases_lag12',
 'active_months_lag12',
 'category_4']

In [58]:
print(f'Train Merged: {len(train_merge)} rows, {len(train_merge["card_id"].unique())} entries\n')
print(train_merge.isnull().sum())

Train Merged: 12572010 rows, 201917 entries

first_active_month                0
card_id                           0
feature_1                         0
feature_2                         0
feature_3                         0
target                            0
authorized_flag                8298
city_id                        8298
category_1                     8298
installments                   8298
category_3                     8298
merchant_category_id           8298
merchant_id                    8298
month_lag                      8298
purchase_amount                8298
purchase_date                  8298
category_2                     8298
state_id                       8298
subsector_id                   8298
merchant_group_id              8298
numerical_1                    8298
numerical_2                    8298
most_recent_sales_range        8298
most_recent_purchases_range    8298
avg_sales_lag3                 8298
avg_purchases_lag3             8298
active_months_lag3 

In [59]:
fill_na(train_merge)
print(train_merge.isnull().sum())

first_active_month             0
card_id                        0
feature_1                      0
feature_2                      0
feature_3                      0
target                         0
authorized_flag                0
city_id                        0
category_1                     0
installments                   0
category_3                     0
merchant_category_id           0
merchant_id                    0
month_lag                      0
purchase_amount                0
purchase_date                  0
category_2                     0
state_id                       0
subsector_id                   0
merchant_group_id              0
numerical_1                    0
numerical_2                    0
most_recent_sales_range        0
most_recent_purchases_range    0
avg_sales_lag3                 0
avg_purchases_lag3             0
active_months_lag3             0
avg_sales_lag6                 0
avg_purchases_lag6             0
active_months_lag6             0
avg_sales_

In [60]:
test_merge = pd.merge(test, df_trans_merch, on='card_id', how='left')
test_merge.columns.tolist()

['first_active_month',
 'card_id',
 'feature_1',
 'feature_2',
 'feature_3',
 'authorized_flag',
 'city_id',
 'category_1',
 'installments',
 'category_3',
 'merchant_category_id',
 'merchant_id',
 'month_lag',
 'purchase_amount',
 'purchase_date',
 'category_2',
 'state_id',
 'subsector_id',
 'merchant_group_id',
 'numerical_1',
 'numerical_2',
 'most_recent_sales_range',
 'most_recent_purchases_range',
 'avg_sales_lag3',
 'avg_purchases_lag3',
 'active_months_lag3',
 'avg_sales_lag6',
 'avg_purchases_lag6',
 'active_months_lag6',
 'avg_sales_lag12',
 'avg_purchases_lag12',
 'active_months_lag12',
 'category_4']

In [61]:
fill_na(test_merge)
print(test_merge.isnull().sum())

first_active_month             0
card_id                        0
feature_1                      0
feature_2                      0
feature_3                      0
authorized_flag                0
city_id                        0
category_1                     0
installments                   0
category_3                     0
merchant_category_id           0
merchant_id                    0
month_lag                      0
purchase_amount                0
purchase_date                  0
category_2                     0
state_id                       0
subsector_id                   0
merchant_group_id              0
numerical_1                    0
numerical_2                    0
most_recent_sales_range        0
most_recent_purchases_range    0
avg_sales_lag3                 0
avg_purchases_lag3             0
active_months_lag3             0
avg_sales_lag6                 0
avg_purchases_lag6             0
active_months_lag6             0
avg_sales_lag12                0
avg_purcha

In [62]:
for feature in ['first_active_month', 'purchase_date']:
    train_merge[feature] = pd.to_datetime(train_merge[feature])
    test_merge[feature] = pd.to_datetime(test_merge[feature])

train_merge['period_to_first_transaction'] = train_merge['purchase_date']-train_merge['first_active_month']
train_merge['period_to_first_transaction'] = ((train_merge['period_to_first_transaction'].dt.days)/30.44) # 30.44 - Average number of days in a month

test_merge['period_to_first_transaction'] = test_merge['purchase_date']-test_merge['first_active_month']
test_merge['period_to_first_transaction'] = ((test_merge['period_to_first_transaction'].dt.days)/30.44) # 30.44 - Average number of days in a month

train_merge.drop(columns={'first_active_month', 'purchase_date'}, axis=1, inplace=True)
test_merge.drop(columns={'first_active_month', 'purchase_date'}, axis=1, inplace=True)

In [63]:
train_merge.loc[train_merge['period_to_first_transaction'] < 0, 'period_to_first_transaction'] = 0
test_merge.loc[test_merge['period_to_first_transaction'] < 0, 'period_to_first_transaction'] = 0

print("Statistics for period_to_first_transaction Train")
stats_train = train_merge["period_to_first_transaction"].describe(percentiles=[0.25, 0.5, 0.75])
print(stats_train, '\n')

print("Statistics for period_to_first_transaction Test")
stats_test = test_merge["period_to_first_transaction"].describe(percentiles=[0.25, 0.5, 0.75])
print(stats_test, '\n')

Statistics for period_to_first_transaction Train
count    1.257201e+07
mean     9.199988e+00
std      8.479139e+00
min      0.000000e+00
25%      3.252300e+00
50%      6.701708e+00
75%      1.245072e+01
max      7.779238e+01
Name: period_to_first_transaction, dtype: float64 

Statistics for period_to_first_transaction Test
count    7.730671e+06
mean     9.303392e+00
std      8.626712e+00
min      0.000000e+00
25%      3.252300e+00
50%      6.734560e+00
75%      1.254928e+01
max      7.729961e+01
Name: period_to_first_transaction, dtype: float64 



In [64]:
def get_mode(series):
    mode = series.mode()
    return mode[0] if not mode.empty else None

train_merged_final = train_merge.groupby('card_id').agg({
    'period_to_first_transaction':'mean',
    'installments':'mean',
    'month_lag':'mean',
    'purchase_amount':'mean',
    'numerical_1':'mean',
    'numerical_2':'mean',
    'avg_sales_lag3':'mean',
    'avg_purchases_lag3':'mean',
    'active_months_lag3':'mean',
    'avg_sales_lag6':'mean',
    'avg_purchases_lag6':'mean',
    'active_months_lag6':'mean',
    'avg_sales_lag12':'mean',
    'avg_purchases_lag12':'mean',
    'active_months_lag12':'mean',
    'target':'mean',
    'feature_1': get_mode,
    'feature_2': get_mode,
    'feature_3': get_mode,
    'authorized_flag': get_mode,
    'category_1': get_mode,
    'category_2': get_mode,
    'category_3': get_mode,
    'category_4': get_mode,
    'city_id': get_mode,
    'state_id': get_mode,
    'subsector_id': get_mode,
    'merchant_id': get_mode,
    'merchant_group_id': get_mode,
    'merchant_category_id': get_mode,
    'most_recent_sales_range': get_mode,
    'most_recent_purchases_range': get_mode
}).reset_index();

print(f'Train Set: {len(train_merged_final)} rows, {len(train_merged_final["card_id"].unique())} entries\n')

Train Set: 201917 rows, 201917 entries



In [65]:
test_merged_final = test_merge.groupby('card_id').agg({
    'period_to_first_transaction':'mean',
    'installments':'mean',
    'month_lag':'mean',
    'purchase_amount':'mean',
    'numerical_1':'mean',
    'numerical_2':'mean',
    'avg_sales_lag3':'mean',
    'avg_purchases_lag3':'mean',
    'active_months_lag3':'mean',
    'avg_sales_lag6':'mean',
    'avg_purchases_lag6':'mean',
    'active_months_lag6':'mean',
    'avg_sales_lag12':'mean',
    'avg_purchases_lag12':'mean',
    'active_months_lag12':'mean',
    'feature_1': get_mode,
    'feature_2': get_mode,
    'feature_3': get_mode,
    'authorized_flag': get_mode,
    'category_1': get_mode,
    'category_2': get_mode,
    'category_3': get_mode,
    'category_4': get_mode,
    'city_id': get_mode,
    'state_id': get_mode,
    'subsector_id': get_mode,
    'merchant_id': get_mode,
    'merchant_group_id': get_mode,
    'merchant_category_id': get_mode,
    'most_recent_sales_range': get_mode,
    'most_recent_purchases_range': get_mode
}).reset_index();

print(f'Test Set: {len(test_merged_final)} rows, {len(test_merged_final["card_id"].unique())} entries\n')

Test Set: 123623 rows, 123623 entries



In [66]:
train_merged_final.to_csv('./train_data.csv', index=False)

In [67]:
test_merged_final.to_csv('./test_data.csv', index=False)

In [68]:
df_train = pd.read_csv('./train_data.csv')
df_train.head()

,card_id,period_to_first_transaction,installments,month_lag,purchase_amount,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,...,category_3,category_4,city_id,state_id,subsector_id,merchant_id,merchant_group_id,merchant_category_id,most_recent_sales_range,most_recent_purchases_range
0,C_ID_00007093c1,7.998978,1.244444,-4.266667,-0.638961,0.190402,0.157132,0.994667,1.013273,3.0,...,B,Y,-1.0,-1,8,M_ID_57df19bf28,71128,755,A,A
1,C_ID_0001506ef0,15.137976,0.018182,-4.254546,-0.492424,1.191807,1.028121,1.052909,1.072006,3.0,...,A,Y,137.0,19,33,M_ID_b1fc88154d,321,705,A,A
2,C_ID_000183fdda,3.192593,1.841270,-2.253968,-0.510138,0.189379,0.134690,1.047778,1.046282,3.0,...,B,Y,161.0,3,16,M_ID_f9cfe0a43b,35,367,B,C
3,C_ID_00027503e2,4.367522,0.000000,-4.184210,-0.741493,-0.004765,-0.006852,0.999211,0.992998,3.0,...,A,N,146.0,19,33,M_ID_8de747a1c2,9207,705,C,C
4,C_ID_0002c7c2c1,6.512668,0.000000,-2.947368,-0.661876,15.560765,14.361757,1.024386,1.053682,3.0,...,A,Y,69.0,9,27,M_ID_f40ccc1312,35,178,A,A


In [69]:
# Convert selected columns to float32 if they exist in the DataFrame
for feature in ['period_to_first_transaction',
                'installments',
                'month_lag',
                'purchase_amount',
                'numerical_1',
                'numerical_2',
                'avg_sales_lag3',
                'avg_purchases_lag3',
                'active_months_lag3',
                'avg_sales_lag6',
                'avg_purchases_lag6',
                'active_months_lag6',
                'avg_sales_lag12',
                'avg_purchases_lag12',
                'active_months_lag12',
                'target',
                'count']:
    if feature in df_train.columns:  # Check if the column exists
        df_train[feature] = df_train[feature].astype('float32')

# Convert selected columns to category if they exist in the DataFrame
for feature in ['feature_1',
                'feature_2',
                'feature_3',
                'authorized_flag',
                'category_1',
                'category_2',
                'category_3',
                'category_4',
                'city_id',
                'state_id',
                'subsector_id',
                'merchant_group_id',
                'merchant_category_id',
                'most_recent_sales_range',
                'most_recent_purchases_range']:
    if feature in df_train.columns:  # Check if the column exists
        df_train[feature] = df_train[feature].astype('category')

# Verify data types
print(df_train.dtypes)

# Separate features and target
X = df_train.drop(columns=['card_id', 'merchant_id', 'target'], errors='ignore')
y = df_train['target']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


card_id                          object
period_to_first_transaction     float32
installments                    float32
month_lag                       float32
purchase_amount                 float32
numerical_1                     float32
numerical_2                     float32
avg_sales_lag3                  float32
avg_purchases_lag3              float32
active_months_lag3              float32
avg_sales_lag6                  float32
avg_purchases_lag6              float32
active_months_lag6              float32
avg_sales_lag12                 float32
avg_purchases_lag12             float32
active_months_lag12             float32
target                          float32
feature_1                      category
feature_2                      category
feature_3                      category
authorized_flag                category
category_1                     category
category_2                     category
category_3                     category
category_4                     category


In [70]:

# Identify categorical columns by checking if they are of 'category' or 'object' dtype
categorical_columns = [col for col in df_train.columns if df_train[col].dtype == 'category' or df_train[col].dtype == 'object']

# Apply encoding based on the cardinality of each categorical column
for col in categorical_columns:
    if df_train[col].nunique() <= 10:  # Low cardinality
        # One-hot encode and drop the original column
        df_train = pd.get_dummies(df_train, columns=[col], drop_first=True)
    else:  # High cardinality
        # Frequency encoding
        freq = df_train[col].value_counts()
        df_train[f'{col}_freq'] = df_train[col].map(freq)
        # Drop the original high cardinality column
        df_train = df_train.drop(columns=[col])

# Verify the DataFrame to ensure all features are now numeric
print(df_train.dtypes)

# Prepare features (X) and target (y) for model training
X = df_train.drop(columns=['card_id', 'merchant_id', 'target'], errors='ignore')
y = df_train['target']



period_to_first_transaction       float32
installments                      float32
month_lag                         float32
purchase_amount                   float32
numerical_1                       float32
numerical_2                       float32
avg_sales_lag3                    float32
avg_purchases_lag3                float32
active_months_lag3                float32
avg_sales_lag6                    float32
avg_purchases_lag6                float32
active_months_lag6                float32
avg_sales_lag12                   float32
avg_purchases_lag12               float32
active_months_lag12               float32
target                            float32
card_id_freq                        int64
feature_1_2                         uint8
feature_1_3                         uint8
feature_1_4                         uint8
feature_1_5                         uint8
feature_2_2                         uint8
feature_2_3                         uint8
feature_3_1                       

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Ensure all columns are numeric after encoding
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')  # Convert non-numeric to NaN if any

# Check for any remaining NaN values (indicating failed conversions)
if X.isnull().sum().sum() > 0:
    print("Warning: Missing values found after conversion, filling with mean values.")
    X.fillna(X.mean(), inplace=True)  # Replace NaNs with column means

# Split data into 80% training and 20% validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Decision Tree Regressor
model = DecisionTreeRegressor(random_state=42, max_depth=3)  # Adjust max_depth as needed

# Fit the model
model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val)

# Calculate Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f'\nRMSE on Test Set: {rmse}')



RMSE on Test Set: 3.7633819499494314
